Reading Data

In [11]:
from pathlib import Path
import pandas as pd

BASE_DIR = Path.cwd().parent.parent
DATA_PATH = BASE_DIR / "media" / "data.xlsx"
df = pd.read_excel(DATA_PATH)
df

,year,month,day,hour,generation,temp,feelslike,dew,humidity,precip,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,2022,6,22,0,0.0,12.4,12.4,9.2,80.83,0.0,...,1007.9,91.1,10.0,0,0.0,0,10,Overcast,cloudy,33177099999.3339
1,2022,6,22,1,0.0,12.5,12.5,9.0,79.23,0.0,...,1007.0,95.7,24.1,0,0.0,0,10,Overcast,cloudy,remote
2,2022,6,22,2,0.0,12.7,12.7,9.0,78.19,0.0,...,1007.0,95.3,24.1,0,0.0,0,10,Overcast,cloudy,remote
3,2022,6,22,3,0.0,10.9,10.9,8.1,82.85,0.0,...,1009.8,80.0,10.0,0,0.0,0,10,Partially cloudy,partly-cloudy-night,33393099999
4,2022,6,22,4,0.0,12.2,12.2,8.8,79.72,0.0,...,1007.0,97.1,20.8,0,0.0,0,10,Overcast,cloudy,remote
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13052,2024,2,23,19,0.0,9.6,7.4,3.8,67.13,0.0,...,1007.0,98.4,24.1,0,0.0,0,10,Overcast,cloudy,NaN
13053,2024,2,23,20,0.0,9.1,6.7,3.7,68.94,0.0,...,1008.0,94.0,24.1,0,0.0,0,10,Overcast,cloudy,NaN
13054,2024,2,23,21,0.0,8.6,6.3,3.4,69.81,0.0,...,1008.0,78.2,24.1,0,0.0,0,10,Partially cloudy,partly-cloudy-night,NaN
13055,2024,2,23,22,0.0,7.9,5.7,3.3,72.70,0.0,...,1009.0,87.1,24.1,0,0.0,0,10,Partially cloudy,partly-cloudy-night,NaN


Checking null values for each column


In [13]:
print(df.isnull().sum())
print(df.shape) #13057 rows present

year                    0
month                   0
day                     0
hour                    0
generation              3
temp                    0
feelslike               0
dew                     0
humidity                0
precip                  0
precipprob              0
preciptype          10375
snow                    0
snowdepth               0
windgust                0
windspeed               0
winddir                 0
sealevelpressure        0
cloudcover              0
visibility              0
solarradiation          0
solarenergy             0
uvindex                 0
severerisk              0
conditions              0
icon                    0
stations              569
dtype: int64
(13057, 27)


 Null values are present in preciptype,stations and generation columns.
 Drop null rows that have no station value.
 Fill null values in preciptype with "None".


In [14]:
df.fillna({"preciptype": "None"}, inplace=True)
df.dropna(inplace=True)

Recheck null values

In [15]:

print(df.isnull().sum())                           # No null values present
print(df.shape)                                    # currently 12485 rows present

year                0
month               0
day                 0
hour                0
generation          0
temp                0
feelslike           0
dew                 0
humidity            0
precip              0
precipprob          0
preciptype          0
snow                0
snowdepth           0
windgust            0
windspeed           0
winddir             0
sealevelpressure    0
cloudcover          0
visibility          0
solarradiation      0
solarenergy         0
uvindex             0
severerisk          0
conditions          0
icon                0
stations            0
dtype: int64
(12485, 27)


 Considering maximum power of each station as system size


In [16]:
system_size = df.groupby("stations")["generation"].max()
df["system size"]=df["stations"].map(system_size)

 Converting negative generation values to zero


In [17]:
df["generation"] = df["generation"].clip(lower=0)


Using normalized power=power/system size


In [18]:
df["normalized power"]=df["generation"]/df["system size"]


Exporting processed data to csv file


In [20]:
df.to_csv(BASE_DIR / "media" / "ProcessedData.csv", index=False)
# final output:predict normalized power using weather data and multiply it with user's system size

In [ ]:
# Yet to work on:
# import pandas as pd
# import pathlib as path

# BASE_DIR=path.Path(__file__).resolve().parent.parent.parent
# DATA_PATH=BASE_DIR/"media"/"ProcessedData.csv"
# df=pd.read_csv(DATA_PATH)
# # Checking out data types of each column and dividing them into categorical and numerical columns

# print(df.dtypes)
# categorical_cols=["preciptype","stations","icon","conditions"]
# numeric_cols = [c for c in df.columns if c not in categorical_cols]
# print(df[numeric_cols].corr())
# print(df["generation"].describe())
# import plotly.express as px
# fig=px.histogram(df, x="generation", nbins=50, title="Generation Distribution")

# fig.update_layout(bargap=0.1)
# fig.show()
#approach
# 1. Understanding relationship between target and numeric features using correlation and scatter plots
# 2. Understanding relationship between target and categorical features using box plots and violin plots
# 3. If categorical features look important to the target we will do one hot encoding for that.
# (if there is only two possible values we can do binary encoding)
# 4. Plot the difference after encoding to see if there is any improvement in correlation
# 5. After all these steps we will standardize the numeric features using StandardScaler from sklearn so that weight is high for the most
#  affecting feature that affects the target the most.
# 6. Now whenever we get a new data point we will do the same transformations on that data point before feeding it to the model.

# few important points
# 1. split data into train test and validation sets according to year
# 2. Imputation
# 3. Scaling
# 4. Encoding

# Remember decision tree uses gini score s